# REST API for PACO server


The docs is available at http://localhost:8080/docs or at [docs](http://localhost:8080/docs) 

In [ ]:
#################
# IMPORTS
#################
import requests
import getpass
import re
import random
import string

#################
# HEADERS
#################
headers = {
    "Content-Type": "application/json",
}
url = "http://127.0.0.1:8000/"

## Define the *BPMN+CPI*

In the following cell the *BPMN+CPI* is defined. 
 - expression: A string representing the BPMN expression, which defines the sequence and parallelism of tasks.
 - impacts: A dictionary where keys are task names and values are lists of impacts (e.g., costs, durations).
 - durations: A dictionary where keys are task names and values are lists representing the duration range [min, max] for each task.
 - probabilities: A dictionary where keys are natures (e.g., 'N1') and values are their probabilities.
 - loop_round: A dictionary for defining loop rounds, if any.
 - names: A dictionary mapping event names to their string representations that are displayed when the bpmn is drawn.
 - delays: A dictionary where keys are event names (e.g., 'C1') and values are their delays.
 - loop_probability: A dictionary for defining loop probabilities, if any.
 - impacts_names: A list of impact names (e.g., ['cost_electricity']).

In [ ]:
bpmn = {
    "expression": "(Cutting, ((Bending, (HP^[N1]LP)) || (Milling, (FD/[C1]RD))), (HPHS / [C2] LPLS))",
    "impacts": {"Cutting": [10, 1], "Bending": [20, 1],
        "Milling": [50, 1], "HP": [5, 4], "LP": [8, 1],
        "FD": [30, 1], "RD": [10, 1], "HPHS": [40, 1],
        "LPLS": [20, 3]
    },
    "durations": {"Cutting": [0, 1], "Bending": [0, 1],
        "Milling": [0, 1], "HP": [0, 2], "LP": [0, 1],
        "FD": [0, 1], "RD": [0, 1], "HPHS": [0, 1],
        "LPLS": [0, 2]},
    "impacts_names": ["electric_energy", "worker hours"], 
    "probabilities": {"N1": 0.2}, 
    "delays": {"C1": 0, "C2": 0},
    "names": {"C1": "C1", "C2": "C2", "N1": "N1"}, 
    "loops_prob" : {}, "loops_round": {}, "h": 0,
}    

### GENERATE A RANDOM BPMN

In [ ]:
###########################
# Define the BPMN STRUCTURE
###########################
expression = '(((((((T1,T2)/[C1]((T3,T4)||T5)),((T6,T7)^[N1]T8)),((T9/[C2]T10),(T11,((T12,T13),T14)))),(((T15/[C3]T16)^[N3]T17)^[N2](T18,T19)))/[C4]((((T20,T21),T22)||T23),((T24,T25)/[C5]T26)))||((T27||((T28^[N4]T29)^[N5](T30/[C6](((T31,T32),((T33^[N7]T34)/[C7]T35)),(T36,T37)))))||T38))'

impacts_names = ["cost", "CO2"]

impacts_range = [1, 50]
duration_range = [1, 100]
delay_range = [0, 10]

tasks = sorted(set(re.findall(r'T\d+', expression)))
natures = sorted(set(re.findall(r'N\d+', expression)))
choices = sorted(set(re.findall(r'C\d+', expression)))
bpmn2 = {
    'expression': expression,
    'impacts': {task: [random.randint(impacts_range[0], impacts_range[1]) for _ in impacts_names] for task in tasks},
    'durations': {task: [1, random.randint(duration_range[0], duration_range[1])] for task in tasks},
    'impacts_names': impacts_names,
    'delays': {choice: random.randint(delay_range[0], delay_range[1]) for choice in choices},
    'probabilities': {nature: round(random.uniform(0.1, 0.9), 2) for nature in natures},
    'names': {choice: choice for choice in choices} | {nature: nature for nature in natures},
    "loops_prob" : {}, "loops_round": {}, "h": 0,
}

In [ ]:
expression

### Check if the expression compliy with the defined grammmar

In [ ]:
resp = requests.get(f'{url}check_correct_process_expression', params={'expression': bpmn['expression']},  headers=headers)
if resp.status_code != 200:
    print('Error in the request', resp.text)
elif resp.text == 'true':
    print('BPMN grammar is correct')
else:
    print('BPMN grammar is incorrect')

### Print Diagram

In [ ]:
import pydot
import graphviz
from IPython.display import display, SVG

data = {
    "bpmn": bpmn, 
}
response = requests.post(f'{url}create_sese_diagram', json=data,  headers=headers)
# Check if response is successful and save the file as a svg
if response.status_code == 200:
    display(SVG(graphviz.Source(response.json()['graph']).pipe(format="svg")))
else:
    print(f"Error: {response.status_code}")
    print(f"Response: {response.text}")

## Calcolate strategy


Remember to choose an appropriate bound.

All the times are in ms

In [ ]:
import json
import gzip

##################################
# STRATEGY CALCULATION + EXPLAINER
##################################
data = {
    'bpmn': bpmn,
    'bound': [120.0, 200.0],
    'algo': 'paco',
}
response = requests.get(
    f'{url}calc_strategy_general',
    json=data,
    headers=headers,
)
response

In [ ]:
json_response = response.json()
with open('client.json', 'w') as f:
    json.dump(json_response, f, indent=4)

In [ ]:
json_response["result"]

In [ ]:
json_response["times"]

In [ ]:
bpmn = json_response["bpmn"]
bpmn

In [ ]:
impacts_names = bpmn["impacts_names"]
impacts_size = len(impacts_names)
impacts_names

In [ ]:
bound = json_response["bound"]
bound

In [ ]:
import graphviz
from IPython.display import display, SVG
from src.paco.parser.parse_tree import ParseTree
parse_tree, pending_choice, pending_natures = ParseTree.from_json(json_response["parse_tree"], impacts_size, 0)
dot =parse_tree.to_dot()
display(SVG(graphviz.Source(dot).pipe(format="svg")))

In [ ]:
expected_impacts = json_response["expected_impacts"]
expected_impacts

In [ ]:
frontier_solution_id = set(map(int, json_response["frontier_solution"].strip("[]").split(",")))
frontier_solution_id

In [ ]:
from src.paco.searcher.create_execution_tree import ExecutionTree

execution_tree = ExecutionTree.from_json(parse_tree, json_response["execution_tree"], bpmn["impacts_names"])

# With frontier node in blue
dot = execution_tree.to_dot(state=True, executed_time=False, diff=True, frontier=frontier_solution_id)

# Without frontier node colored
#dot = execution_tree.to_dot(state=True, executed_time=False, diff=True)
display(SVG(graphviz.Source(dot).pipe(format="svg")))

dot = execution_tree.to_dot(state=True, executed_time=True, diff=True, frontier=frontier_solution_id)
display(SVG(graphviz.Source(dot).pipe(format="svg")))

dot = execution_tree.to_dot(state=True, executed_time=True, diff=False, frontier=frontier_solution_id)
display(SVG(graphviz.Source(dot).pipe(format="svg")))

dot = execution_tree.to_dot(state=False, executed_time=True, diff=True, frontier=frontier_solution_id)
display(SVG(graphviz.Source(dot).pipe(format="svg")))

In [ ]:
json_response["expected_impacts"]

In [ ]:
json_response["frontier_solution"] #ExecutionViewPoint Id

In [ ]:
import graphviz
from paco.explainer.bdd.bdds import bdds_from_json
from IPython.display import display, SVG

explained_choices = bdds_from_json(parse_tree, json_response["bdds"])
for choice, bdd in explained_choices.items():
    print(choice.name)
    svg_data = graphviz.Source(bdd.bdd_to_dot()).pipe(format="svg")
    display(SVG(svg_data))

In [ ]:
from src.paco.execution_tree.execution_tree import ExecutionTree

strategy_tree = ExecutionTree.from_json(parse_tree, json_response["strategy_tree"], impacts_names, explained_choices)

#TREE_STATE
dot = strategy_tree.to_dot(state=True, executed_time=False, diff=True)
display(SVG(graphviz.Source(dot).pipe(format="svg")))

#TREE_STATE_TIME
dot = strategy_tree.to_dot(state=True, executed_time=True, diff=True)
display(SVG(graphviz.Source(dot).pipe(format="svg")))

#TREE_STATE_TIME_EXTENDED
dot = strategy_tree.to_dot(state=True, executed_time=True, diff=False)
display(SVG(graphviz.Source(dot).pipe(format="svg")))

#TREE_TIME
dot = strategy_tree.to_dot(state=False, executed_time=True, diff=True)
display(SVG(graphviz.Source(dot).pipe(format="svg")))

### Strategy step-by-step

In [ ]:
############################################
# STEP 1: Create the execution & parse tree
############################################

response = requests.get(
	f'{url}create_execution_tree',
	json=bpmn,
	headers=headers,
)

if response.status_code == 200:
	print("Execution tree created successfully")
	print(response.json().keys())
	parse_tree = ParseTree.from_json(response.json()['parse_tree'], len(bpmn[IMPACTS_NAMES]), 0)
	execution_tree = ExecutionTree.from_json(parse_tree, response.json()['execution_tree'], bpmn[IMPACTS_NAMES])

else:
	print(f"Error: {response.status_code}, {response.text}")


In [ ]:
###############################
# STEP 2: Search for a strategy
###############################
'''not working'''
data = {
    "impacts_names" : bpmn['impacts_names'],
    "execution_tree": execution_tree,
    "bound" : [20.0, 20.0],
    "search_only": True,
}
response = requests.get(
    f'{url}search_only_strategy',
    json=data,
    headers=headers,
)

if response.status_code == 200:
    strategy = response.json()['strategy']
    for key, value in strat.json().items():
        print(f"{key}: {value}")
else:
    print(f"Error: {response.status_code}, {response.text}")

In [ ]:
##################################
# GET PARSE TREE
##################################

response = requests.get(
    f'{url}get_parse_tree',
    json=bpmn,
    headers=headers,
)
if response.status_code == 200:
    parse_tree = response.json()
    print("Parse tree retrieved successfully")
else:
    print(f"Error: {response.status_code}, {response.text}")

The different type of explainer:
- CURRENT_IMPACTS = 0
- UNAVOIDABLE_IMPACTS = 1
- DECISION_BASED = 2
- HYBRID = 3 (default)

In [ ]:
############################################################
# STEP 3: Create the explainer (if choice list is not empty)
############################################################
'''not working'''
data = {
    "parse_tree" : parse_tree,
    "impacts_names" : bpmn['impacts_names'],
    "type_explainer": 2,
    "strategy": strategy,
}
response = requests.get(
    f'{url}explainer',
    json=data,
    headers=headers,
)

if response.status_code == 200:
    for key, value in response.json().items():
        print(f"{key}: {value}")
else:
    print(f"Error: {response.status_code}, {response.text}")


### Visualize the trees

#### PARSE TREE

In [ ]:
response = requests.get(f'{url}get_execution_tree_state', headers=headers)
if response.status_code == 200:
    display(SVG(response.content))
else:
    print(f"Error: {response.status_code}, {response.text}")

#### EXCECUTION TREES

In [ ]:
response = requests.get(f'{url}get_execution_tree_state', headers=headers)
if response.status_code == 200:
    display(SVG(response.content))
else:
    print(f"Error: {response.status_code}, {response.text}")

In [ ]:
response = requests.get(f'{url}get_execution_tree_state_time', headers=headers)
if response.status_code == 200:
    display(SVG(response.content))
else:
    print(f"Error: {response.status_code}, {response.text}")

In [ ]:
response = requests.get(f'{url}get_execution_tree_state_time_extended', headers=headers)
if response.status_code == 200:
    display(SVG(response.content))
else:
    print(f"Error: {response.status_code}, {response.text}")

In [ ]:
response = requests.get(f'{url}get_execution_tree_time', headers=headers)
if response.status_code == 200:
   display(SVG(response.content))
else:
    print(f"Error: {response.status_code}, {response.text}")

#### STRATEGY TREES

In [ ]:
response = requests.get(f'{url}get_strategy_tree_state', headers=headers)
if response.status_code == 200:
    display(SVG(response.content))
else:
    print(f"Error: {response.status_code}, {response.text}")

In [ ]:
response = requests.get(f'{url}get_strategy_tree_state_time', headers=headers)
if response.status_code == 200:
    display(SVG(response.content))
else:
    print(f"Error: {response.status_code}, {response.text}")

In [ ]:
response = requests.get(f'{url}get_strategy_tree_state_time_extended', headers=headers)
if response.status_code == 200:
    display(SVG(response.content))
else:
    print(f"Error: {response.status_code}, {response.text}")

In [ ]:
response = requests.get(f'{url}get_strategy_tree_time', headers=headers)
if response.status_code == 200:
    display(SVG(response.content))
else:
    print(f"Error: {response.status_code}, {response.text}")

#### EXPLAINER TREES

In [ ]:
response = requests.get(f'{url}get_explainer_decision_tree', headers=headers)
if response.status_code == 200:
    display(SVG(response.content))
else:
    print(f"Error: {response.status_code}, {response.text}")

In [ ]:
response = requests.get(f'{url}get_explainer_bdd', headers=headers)
if response.status_code == 200:
    display(SVG(response.content))
else:
    print(f"Error: {response.status_code}, {response.text}")

## LLM

### Chat with the LMM 

In [ ]:
session_id = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(8))
##################################
# SET THE DATA FOR THE LLM
##################################
data = {
    "session_id": session_id,
    "url": input("Enter the URL of the model: "),
    "api_key": getpass.getpass("Enter the API key: "),
    "model": input("Enter the the model: "),
    "temperature": 0.7,
    "verbose": False,
}

Define the prompt

In [ ]:
# If wanted can be used also this predefined prompt that consituates the example found in the paper
# prompt = '''
# Now I have to complete the writing task before 
# having a nature between talking with the publisher or to print the page written.
# Then, I choose between going to the coffee or go to the gym.
# '''
prompt = input("Enter your prompt: ")

In [ ]:
data["prompt"] = prompt
response = requests.post(f'{url}invoke_agent', headers=headers, json=data)

if response.status_code == 200:
    print(response.json()['response'])
else:
    print(f"Error: {response.status_code}")
    print(f"Response: {response.text}")

### Get Chat History

In [ ]:
response = requests.get(f'{url}get_chat_history', headers=headers, params={"session_id": session_id})

if response.status_code == 200:
    for message in response.json():
        if message["role"] == "human":
            print(f"User: {message['content']}")
        elif message["role"] == "ai":
            print(f"Assistant: {message['content']}")
            print("\n")
            
else:
    print(f"Error: {response.status_code}")
    print(f"Response: {response.text}")